In [1]:
%env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd #其實不會用到

from ipywidgets import interact, IntSlider, Button

In [3]:
# Keras functions
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

# Keras dataset
from keras.datasets import mnist

# Keras utils
from keras.utils import np_utils

c:\users\user\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
(x_train0, y_train0), (x_test0, y_test0) = mnist.load_data()

In [7]:
x_train = x_train0.reshape(60000, 28*28)
x_test = x_test0.reshape(10000, 28*28)

In [8]:
x_train -= x_train.min()
x_train = x_train/x_train.max()

y_train = np_utils.to_categorical(y_train0, 10)
y_test = np_utils.to_categorical(y_test0, 10)

In [19]:
y_train01 = np.ones_like(y_train0)
y_train01[y_train0%2==0] = 0

y_test01 = np.ones_like(y_test0)
y_test01[y_test01%2==0] = 0

y_train01 = np_utils.to_categorical(y_train01, 2)
y_test01 = np_utils.to_categorical(y_test01, 2)

## 辨識0-9

In [36]:
all_except_last = [Dense(500, input_dim=784), Activation('sigmoid'), Dense(500), Activation('sigmoid')]

In [37]:
model_num = Sequential(all_except_last + [Dense(10), Activation('softmax')])
model_num.compile(optimizer=SGD(lr=0.09), loss='categorical_crossentropy', metrics=['acc'])

In [38]:
model_num.fit(x_train, y_train, 
          batch_size=100, 
          epochs=5, 
          verbose=1, 
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 13s 223us/step - loss: 1.4932 - acc: 0.5335 - val_loss: 0.6877 - val_acc: 0.7921
Epoch 2/5
60000/60000 [==============================] - 12s 197us/step - loss: 0.5322 - acc: 0.8523 - val_loss: 0.6054 - val_acc: 0.8561
Epoch 3/5
60000/60000 [==============================] - 14s 237us/step - loss: 0.4023 - acc: 0.8861 - val_loss: 0.5967 - val_acc: 0.8673
Epoch 4/5
60000/60000 [==============================] - 16s 270us/step - loss: 0.3582 - acc: 0.8957 - val_loss: 0.5476 - val_acc: 0.8823
Epoch 5/5
60000/60000 [==============================] - 13s 215us/step - loss: 0.3350 - acc: 0.9027 - val_loss: 0.5361 - val_acc: 0.8894


## 用轉移學習辨識奇偶數

In [39]:
model_eo = Sequential(all_except_last + [Dense(2), Activation('softmax')])
model_eo.compile(optimizer=SGD(lr=0.09), loss='categorical_crossentropy', metrics=['acc'])
#model_eo.evaluate(x_train, y_train01, batch_size=10000)

In [40]:
training_history = model_eo.fit(x_train, y_train01, batch_size=1000, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 7s 109us/step - loss: 0.8414 - acc: 0.7089
Epoch 2/5
60000/60000 [==============================] - 5s 91us/step - loss: 0.3326 - acc: 0.8547
Epoch 3/5
60000/60000 [==============================] - 5s 88us/step - loss: 0.3167 - acc: 0.8631
Epoch 4/5
60000/60000 [==============================] - 5s 89us/step - loss: 0.3111 - acc: 0.8665
Epoch 5/5
60000/60000 [==============================] - 5s 89us/step - loss: 0.3011 - acc: 0.8710


In [41]:
model_eo.evaluate(x_train, y_train01, batch_size=1000)

60000/60000 [==============================] - 3s 52us/step


[0.29319584319988884, 0.8754499981800715]

In [ ]:
for layer in all_except_last: # 借來的模型權重在訓練當中不要被改變
    layer.trainable = False

## Functional API

In [44]:
from keras.models import Model
from keras.layers import Input
from keras.layers import concatenate, add

In [70]:
f_1 = Dense(500, activation='sigmoid')
f_2 = Dense(500, activation='sigmoid')
f_3 = Dense(10, activation='softmax')
f_4 = Dense(500, activation='relu')

In [72]:
x = Input(shape=(784,))

h_1 = f_1(x)
h_2 = f_2(h_1)
z = f_4(h_1)
u = concatenate([h_2, z])

y = f_3(u)

In [73]:
model = Model(x, y)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 784)          0                                            
__________________________________________________________________________________________________
dense_55 (Dense)                (None, 500)          392500      input_7[0][0]                    
__________________________________________________________________________________________________
dense_56 (Dense)                (None, 500)          250500      dense_55[1][0]                   
__________________________________________________________________________________________________
dense_58 (Dense)                (None, 500)          250500      dense_55[1][0]                   
__________________________________________________________________________________________________
concatenat

In [74]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.09), metrics=['accuracy'])

In [75]:
model.fit(x_train, y_train, batch_size=100, epochs=5, validation_data=(x_test, y_test), verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 20s 333us/step - loss: 0.9230 - acc: 0.6986 - val_loss: 0.7340 - val_acc: 0.8420
Epoch 2/5
60000/60000 [==============================] - 21s 347us/step - loss: 0.3671 - acc: 0.8908 - val_loss: 0.5567 - val_acc: 0.8871
Epoch 3/5
60000/60000 [==============================] - 19s 321us/step - loss: 0.3134 - acc: 0.9075 - val_loss: 0.4775 - val_acc: 0.9036
Epoch 4/5
60000/60000 [==============================] - 16s 271us/step - loss: 0.2813 - acc: 0.9169 - val_loss: 0.4419 - val_acc: 0.9091
Epoch 5/5
60000/60000 [==============================] - 17s 288us/step - loss: 0.2558 - acc: 0.9237 - val_loss: 0.3402 - val_acc: 0.9284


In [76]:
model.evaluate(x_test, y_test, batch_size=10000)

10000/10000 [==============================] - 0s 47us/step


[0.3401852548122406, 0.9283999800682068]